In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
import numpy as np

### 1. Data Loading

In [16]:
file_path = "/Users/zhangnuonan/Desktop/Second Semester/Machine Learning/ML project/cleaned table/combined data/4.20_combined_all.csv"
df = pd.read_csv(file_path)

### 2. Quality Checking

In [18]:
print(df.describe())

         subject_id        hadm_id  hospital_expire_flag     icustay_id  \
count  62925.000000   62925.000000          62925.000000   61725.000000   
mean   33773.396583  149952.549940              0.106222  249968.059976   
std    28092.688727   28896.524028              0.308124   28892.344985   
min        2.000000  100001.000000              0.000000  200001.000000   
25%    11994.000000  124923.000000              0.000000  224933.000000   
50%    24170.000000  149943.000000              0.000000  249943.000000   
75%    53875.000000  174962.000000              0.000000  274984.000000   
max    99999.000000  199999.000000              1.000000  299999.000000   

                los  mechanical_ventilation  VASOPRESSOR_USE  
count  61725.000000            61725.000000     61725.000000  
mean       4.937720                0.889089         0.185322  
std        9.649622                0.314025         0.388562  
min        0.000100                0.000000         0.000000  
25%      

In [20]:
print(df.isnull().sum())

subject_id                    0
gender                        0
Dob                           0
hadm_id                       0
admittime                     0
dischtime                     0
deathtime                 56241
admission_type                0
admission_location            0
insurance                     0
language                  26879
religion                    482
marital_status            10392
ethnicity                     0
hospital_expire_flag          0
icustay_id                 1200
icu_intime                 1200
icu_outtime                1200
los                        1200
mechanical_ventilation     1200
VASOPRESSOR_USE            1200
dtype: int64


### 3. Reshape Dataframe

In [22]:
df['Dob'] = pd.to_datetime(df['Dob'], errors='coerce')
df['admittime'] = pd.to_datetime(df['admittime'], errors='coerce')
df['dischtime'] = pd.to_datetime(df['dischtime'], errors='coerce')
df['icu_intime'] = pd.to_datetime(df['icu_intime'], errors='coerce')
df['deathtime'] = pd.to_datetime(df['deathtime'], errors='coerce')

df['adm_age'] = df.apply(
    lambda row: ((row['admittime'].timestamp() - row['Dob'].timestamp()) / (365.25 * 24 * 3600)) 
    if pd.notna(row['admittime']) and pd.notna(row['Dob']) else np.nan, 
    axis=1
).round(1)

df['los_adm'] = df.apply(
    lambda row: ((row['dischtime'].timestamp() - row['admittime'].timestamp()) / (24 * 3600)) 
    if pd.notna(row['dischtime']) and pd.notna(row['admittime']) else np.nan, 
    axis=1
).round(4)

df['icu_age'] = df.apply(
    lambda row: ((row['icu_intime'].timestamp() - row['Dob'].timestamp()) / (365.25 * 24 * 3600)) 
    if pd.notna(row['icu_intime']) and pd.notna(row['Dob']) else np.nan, 
    axis=1
).round(1)

df['death_age'] = df.apply(
    lambda row: ((row['deathtime'].timestamp() - row['Dob'].timestamp()) / (365.25 * 24 * 3600)) 
    if pd.notna(row['deathtime']) and pd.notna(row['Dob']) else np.nan, 
    axis=1
).round(1)


# deal with negative ages and lengths of stay
df.loc[df['adm_age'] < 0, 'admission_age'] = np.nan
df.loc[df['los_adm'] < 0, 'length_of_stay'] = np.nan
df.loc[df['icu_age'] < 0, 'icu_age'] = np.nan
df.loc[df['death_age'] < 0, 'death_age'] = np.nan

# summarize by subject_id
aggregated_df = df.sort_values(['subject_id', 'admittime', 'icu_intime'], ascending=[True, True, True]) \
                  .groupby('subject_id').agg({
    'gender': 'last',               
    'insurance': 'last',            
    'language': 'last',            
    'religion': 'last',             
    'marital_status': 'last',       
    'ethnicity': 'last',            
    'hadm_id': 'count',             
    'adm_age': 'last',               
    'los_adm': 'last',              
    'admission_location': 'last',   
    'admission_type': 'last',       
    'admission_age': 'last',       
    'icustay_id': 'count',         
    'icu_age': 'last',            
    'los': 'last',                  
    'mechanical_ventilation': 'max',  
    'VASOPRESSOR_USE': 'max',        
    'death_age' : 'last',                
    'hospital_expire_flag': 'last' 
}).reset_index()

aggregated_df.rename(columns={
    'hadm_id': 'admission_count',
    'icustay_id': 'icu_count',
    'hospital_expire_flag': 'hospital_expired',
    'los': 'los_icu',
    'admission_location':'last_admission_location',
    'admission_type':'last_admission_type',
    'los_icu': 'last_los_icu'
    
}, inplace=True)

print(aggregated_df.head())


   subject_id gender insurance language           religion marital_status  \
0           2      M   Private     None      NOT SPECIFIED           None   
1           3      M  Medicare     None           CATHOLIC        MARRIED   
2           4      F   Private     None  PROTESTANT QUAKER         SINGLE   
3           5      M   Private     None           BUDDHIST           None   
4           6      F  Medicare     ENGL      NOT SPECIFIED        MARRIED   

  ethnicity  admission_count  adm_age  los_adm    last_admission_location  \
0     ASIAN                1      0.0   3.8639  PHYS REFERRAL/NORMAL DELI   
1     WHITE                1     76.5  10.7847       EMERGENCY ROOM ADMIT   
2     WHITE                1     47.8   7.7590       EMERGENCY ROOM ADMIT   
3     ASIAN                1      0.0   2.3222  PHYS REFERRAL/NORMAL DELI   
4     WHITE                1     65.9  16.3646  PHYS REFERRAL/NORMAL DELI   

  last_admission_type  admission_age  icu_count  icu_age  los_icu  \
0    

In [24]:
aggregated_df.to_csv('final_patient_data.csv', index=False)